In [4]:
import pandas as pd
import datetime

In [5]:
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet")

# REGLAS CONTAR VP

In [ ]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_week = lost_by_week


,store_name,product_ref,Semana,ano,mes,dia,lost_sales,veces_vp
26475,SUCURSAL CALLE 6,BCS00298125,2025-07-28,2025,7,28,1.0,1
2618,PRINCIPAL COTA,BHE00826125,2025-08-25,2025,8,25,1.0,1
20489,SUCURSAL BUCARAMANGA,DAE04023025,2025-02-10,2025,2,10,1.0,1
15123,SUCURSAL BARRANQUILLA,DAP09109025,2025-03-03,2025,3,3,2.0,2
11289,SUCURSAL BARRANQUILLA,BCS20364125,2025-09-15,2025,9,15,1.0,1
19939,SUCURSAL BUCARAMANGA,DAB09178025,2025-01-20,2025,1,20,1.0,1
34723,SUCURSAL MEDELLIN,DAB08069025,2025-05-19,2025,5,19,1.0,1
42212,SUCURSAL VALLADOLID,BLS00029125,2025-02-10,2025,2,10,3.0,1
11978,SUCURSAL BARRANQUILLA,BHS00606125,2025-04-07,2025,4,7,1.0,1
33972,SUCURSAL MEDELLIN,BLS00068125,2025-07-14,2025,7,14,1.0,1


In [8]:
vp_week.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_año.xlsx")

In [11]:
mes = "TODOS"
vp = vp_week[(vp_week["ano"]==2025)]
vp.to_excel(f"/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_meses/vp_definitivasparaanalisis{mes}.xlsx")


In [56]:
df_vp.sample(10)

,cantidad_reservada,product_ref,cantidad,almacenamiento_tipo,fecha,cantidad_existencia,origen,store_name,store_id_num,cliente_name,cliente_id_num,ventas_perdidas,Semana,ano,mes,dia
45201,0,DAB04772025,1,agotado,2025-07-29,0,cotizacion,SUCURSAL MEDELLIN,8,[CO8914106359] CATERCOL S.A.,3030,1.0,2025-07-28,2025,7,28
9629,0,DLS00264004,1,agotado,2025-01-28,0,cotizacion,SUCURSAL MEDELLIN,8,[CC222222222224] CONSUMIDOR FINAL ITAGUI,27634,1.0,2025-01-27,2025,1,27
21925,1,DCE00796189,1,agotado,2025-03-20,0,cotizacion,SUCURSAL NORTE,7,[CC222222222228] CONSUMIDOR FINAL NORTE,27642,0.0,2025-03-17,2025,3,17
52878,6,DCE26955138,6,agotado,2025-09-03,0,cotizacion,PRINCIPAL COTA,1,"[CO80009296] PROMOCION BALDWIN48%, RACOR46% >...",13659,0.0,2025-09-01,2025,9,1
51969,0,BCS00284125,1,agotado,2025-09-01,0,cotizacion,PRINCIPAL COTA,1,[CC40432669] ANGELICA MARIA CASALLAS VELASQUEZ,11962,1.0,2025-09-01,2025,9,1
40804,0,DAR12276025,1,agotado,2025-07-04,0,cotizacion,SUCURSAL MEDELLIN,8,[CC222222222224] CONSUMIDOR FINAL ITAGUI,27634,1.0,2025-06-30,2025,6,30
13982,0,DAB02887025,1,agotado,2025-02-17,0,cotizacion,PRINCIPAL COTA,1,[CO8000914186] AGROFILTER BOYACA SAS,3404,1.0,2025-02-17,2025,2,17
46039,0,BHE00733125,1,agotado,2025-08-01,0,cotizacion,SUCURSAL BUCARAMANGA,11,[CC91531855] ANDRES MAURICIO SERRANO GUIZA,21530,1.0,2025-07-28,2025,7,28
12958,0,BLS00645125,1,agotado,2025-02-12,0,cotizacion,SUCURSAL BARRANQUILLA,10,[CO9003515420] DISTRIBUIDORA E INVERSIONES LUB...,32923,1.0,2025-02-10,2025,2,10
11046,0,BLE00709125,9,agotado,2025-02-04,4,cotizacion,SUCURSAL VALLADOLID,6,"[CO80009296] PROMOCION BALDWIN48%, RACOR46% >...",13659,5.0,2025-02-03,2025,2,3


In [53]:
df_vp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57686 entries, 0 to 57699
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   cantidad_reservada   57686 non-null  int64         
 1   product_ref          57686 non-null  object        
 2   cantidad             57686 non-null  int64         
 3   almacenamiento_tipo  57686 non-null  object        
 4   fecha                57686 non-null  datetime64[ns]
 5   cantidad_existencia  57686 non-null  int64         
 6   origen               57686 non-null  object        
 7   store_name           57686 non-null  object        
 8   store_id_num         57686 non-null  int64         
 9   cliente_name         57686 non-null  object        
 10  cliente_id_num       57686 non-null  int64         
 11  ventas_perdidas      57686 non-null  float64       
 12  Semana               57686 non-null  datetime64[ns]
 13  ano                  57686 non-null 